In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [ ]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do.

In [ ]:
SELECT membercost
FROM Facilities
WHERE membercost > 0;

In [ ]:
#Q2: How many facilities do not charge a fee to members?

In [ ]:
SELECT COUNT(membercost < 0)
FROM Facilities

In [ ]:
#Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < .2 * monthlymaintenance;

In [ ]:
#Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator.

In [ ]:
SELECT *
FROM Facilities
WHERE facid IN (1, 5);

In [ ]:
#Q5: Produce a list of facilities, with each labelled as'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. 
#Return the name and monthly maintenance of the facilities in question.

In [ ]:
SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive'
ELSE 'cheap' END AS cost
FROM Facilities

In [ ]:
#Q6: You'd like to get the first and last name of the last member(s) who signed up. 
#Try not to use the LIMIT clause for your solution.

In [ ]:
SELECT firstname, surname, MAX(joindate)
FROM Members

In [ ]:
#Q7: Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member formatted as a single column. 
#Ensure no duplicate data, and order by the member name.

In [ ]:
SELECT DISTINCT ( CONCAT( `firstname` , ' ', `surname` )) AS MemberName, f.name 
FROM Members AS m 
INNER JOIN Bookings AS b 
ON m.memid = b.memid 
INNER JOIN Facilities AS f 
ON b.facid = f.facid 
WHERE f.name LIKE '%ennis Cou%' 
ORDER BY MemberName;

In [ ]:
#Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. 
#Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. #Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries.

In [ ]:
SELECT f.name AS FacilityName, CONCAT( m.firstname, ' ', m.surname ) AS MemberName, CASE WHEN f.guestcost * b.slots >30 THEN f.guestcost * b.slots 
WHEN f.membercost * b.slots >30 THEN f.membercost * b.slots 
END AS cost FROM Facilities AS f 
INNER JOIN Bookings AS b 
ON f.facid = b.facid 
INNER JOIN Members AS m ON b.memid = m.memid 
WHERE b.starttime LIKE '2012-09-14%' AND f.membercost * b.slots >0 AND f.guestcost * b.slots >0 
ORDER BY cost DESC;

In [ ]:
#Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. 
#Remember that there's a different cost for guests and members! 

In [ ]:
SELECT sub.name, SUM( sub.revenue ) AS revenue
FROM (
SELECT b.facid, b.memid, f.name, f.guestcost, f.membercost, COUNT( b.facid ) AS facid_count,
CASE
WHEN b.memid =0
THEN COUNT( b.facid ) * f.guestcost
ELSE COUNT( b.facid ) * f.membercost
END AS 'revenue'
FROM Bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid, b.memid
) AS sub
GROUP BY sub.facid
HAVING revenue <=1000;

In [ ]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [ ]:
SELECT DISTINCT ( CONCAT( `surname` , ' ', `firstname` )) AS member_name, recommendedby
FROM Members AS m
WHERE recommendedby > 0
ORDER BY member_name 

In [ ]:
#Q12: Find the facilities with their usage by member, but not guests

In [ ]:
SELECT b.facid, COUNT( b.memid ) AS mem_usage, f.name
FROM (
SELECT facid, memid
FROM Bookings
WHERE memid !=0
) AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid;

In [ ]:
#Q13: Find the facilities usage by month, but not guests

In [ ]:
SELECT b.months, COUNT( b.memid ) AS mem_usage
FROM (
SELECT MONTH( starttime ) AS months, memid
FROM Bookings
WHERE memid !=0
) AS b
GROUP BY b.months;